In [6]:
import os
import pandas as pd

In [7]:
input_folder = "converted csv files"
output_folder = "cleaned csv files"

for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        year = file[-8:-4]
        filepath = os.path.join(input_folder, file)

        df = pd.read_csv(filepath, dtype={"ZIP": str}, parse_dates=["Date"])
        df = df[['ZIP', 'MSA', 'MSA.Title', 'TEMP.Mean', 'HeatIndex.Mean', 'Date']]

        df_avg = (
            df
            .groupby([df['ZIP'], df['Date'].dt.month])
            .agg({
                'MSA': 'first',
                'MSA.Title': 'first',
                'TEMP.Mean': 'mean',
                'HeatIndex.Mean': 'mean'
            })
            .reset_index()
            .rename(columns={'Date': 'Month', 'TEMP.Mean': 'Avg.Temp.Mean', 'HeatIndex.Mean': 'Avg.HeatIndex.Mean'})
        )
        
        df_avg['Avg.Temp.Mean'] = df_avg['Avg.Temp.Mean'] * 9/5 + 32
        df_avg['Avg.HeatIndex.Mean'] = df_avg['Avg.HeatIndex.Mean'] * 9/5 + 32
        
        df.to_csv(os.path.join(output_folder, f"cleaned_climate_{year}.csv"), index=False)
        df_avg.to_csv(os.path.join(output_folder, f"monthly_avg_climate{year}.csv"), index=False)

In [8]:
folder_path = "cleaned csv files"
all_years = []

for file in os.listdir(folder_path):
    if file.startswith("monthly_avg_climate") and file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path, dtype={'ZIP': str})
        all_years.append(df)

In [9]:
combined_df = pd.concat(all_years, ignore_index=True)

In [11]:
ten_year_avg_2008_2017 = (
    combined_df
    .groupby(['ZIP', 'Month'], as_index=False)
    .agg({
        'MSA': 'first',
        'MSA.Title': 'first',
        'Avg.Temp.Mean': 'mean',
        'Avg.HeatIndex.Mean': 'mean'
        })
) 
state_extracted = ten_year_avg_2008_2017['MSA.Title'].str.rsplit(', ', n=1).str[-1]
ten_year_avg_2008_2017['StateList'] = state_extracted.str.split('-')
ten_year_avg_2008_2017 = ten_year_avg_2008_2017.explode('StateList').rename(columns={'StateList': 'State'})
ten_year_avg_2008_2017['Metro'] = ten_year_avg_2008_2017['MSA.Title'].str.rsplit(', ', n=1).str[0]

ten_year_avg_2008_2017.to_csv('ten_year_avg_2008_2017.csv', index=False)

In [4]:
import glob
import pandas as pd
files = glob.glob("cleaned csv files/*.csv")
print(f"Found {len(files)} files to check")

processed = 0
for path in files:
    df = pd.read_csv(path, dtype={'ZIP': str})
    null_count = df.isnull().sum().sum()
    assert null_count == 0, f"Found {null_count} nulls in {path}"
    processed += 1

assert processed == len(files),(
    f"Processed {processed} files but expected {len(files)}"
)
print("All files read and checked")

Found 20 files to check
All files read and checked
